# 02. Magpie Data Generation
## Synthetic Instruction Tuner - Week 1 Day 3-5

This notebook generates synthetic instruction-response pairs using the Magpie method:
1. Load Llama-3.1-8B-Instruct model
2. Generate instructions using template-only prompts
3. Generate responses for each instruction
4. Save checkpoints periodically

**Expected runtime**:
- T4: 16-17 hours → split across 3 days (500 samples/day, 12-hour limit workaround)
- A100: 6-8 hours → complete in single session (no runtime limits)

**Checkpoint Strategy**:
- T4: Checkpoint every 20 samples (~18 min intervals) for safety
- A100: Checkpoint every 100 samples (reduced disk I/O overhead)

**A100 benefits**:
- Complete entire 1,500 sample generation in one session
- Reduced checkpoint frequency minimizes disk I/O
- No need to manually resume across multiple days

**Tip for A100**: Can complete full dataset in one run without worrying about 12h limit

## 1. Setup

In [2]:
# Mount Google Drive
from google.colab import drive
from google.colab import userdata
from huggingface_hub import login

drive.mount('/content/drive')

login(token=userdata.get('HF_TOKEN'))

# Project path
PROJECT_ROOT = "/content/drive/MyDrive/synthetic-instruction-tuner"

Mounted at /content/drive


In [3]:
# Load configuration
import json

with open(f"{PROJECT_ROOT}/config.json", 'r') as f:
    config = json.load(f)

TARGET_SAMPLES = config['data_generation']['target_raw_samples']
CHECKPOINT_INTERVAL = config['data_generation']['checkpoint_interval']

print(f"Configuration loaded!")
print(f"Target samples: {TARGET_SAMPLES}")
print(f"Checkpoint interval: {CHECKPOINT_INTERVAL}")

Configuration loaded!
Target samples: 1500
Checkpoint interval: 20


In [4]:
# Install libraries with latest compatible versions (avoid dependency conflicts)
!pip install -q --upgrade transformers>=4.41.0 peft>=0.7.0 trl>=0.7.4 accelerate>=0.25.0 bitsandbytes>=0.41.3 datasets>=2.16.0

print("✅ Libraries installed successfully!")

✅ Libraries installed successfully!


In [5]:
import torch
import json
import os
from datetime import datetime
from tqdm import tqdm
import random
import gc

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4


## 2. Load Model

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_ID = config['models']['data_generation']

# 4-bit quantization for memory efficiency
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

print(f"Loading {MODEL_ID}...")

Loading meta-llama/Llama-3.1-8B-Instruct...


In [7]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

print(f"Tokenizer loaded. Vocab size: {tokenizer.vocab_size}")

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Tokenizer loaded. Vocab size: 128000


In [8]:
from transformers import AutoModelForCausalLM

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)
model.eval()

print(f"Model loaded!")
print(f"GPU Memory: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Model loaded!
GPU Memory: 5.71 GB


## 3. Magpie Generator Class

In [9]:
class MagpieGenerator:
    """Generate synthetic instructions and responses using Magpie method."""

    def __init__(self, model, tokenizer, config):
        self.model = model
        self.tokenizer = tokenizer
        self.config = config

        # Magpie template (Llama 3.1 format)
        self.instruction_template = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n"
        self.response_template = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

        # Generation settings
        self.temperature = config['data_generation']['temperature']
        self.max_new_tokens = config['data_generation']['max_new_tokens']

    def generate_instruction(self):
        """Generate a single instruction using Magpie method."""
        inputs = self.tokenizer(
            self.instruction_template,
            return_tensors="pt"
        ).to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=256,
                temperature=self.temperature,
                do_sample=True,
                top_p=0.95,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=[
                    self.tokenizer.convert_tokens_to_ids("<|eot_id|>"),
                    self.tokenizer.eos_token_id
                ]
            )

        generated = self.tokenizer.decode(outputs[0], skip_special_tokens=False)
        instruction = self._parse_instruction(generated)
        return instruction

    def generate_response(self, instruction):
        """Generate a response for the given instruction."""
        # Format as conversation
        prompt = f"{self.instruction_template}{instruction}{self.response_template}"

        inputs = self.tokenizer(
            prompt,
            return_tensors="pt"
        ).to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=self.max_new_tokens,
                temperature=0.7,  # Lower temperature for more focused responses
                do_sample=True,
                top_p=0.9,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=[
                    self.tokenizer.convert_tokens_to_ids("<|eot_id|>"),
                    self.tokenizer.eos_token_id
                ]
            )

        generated = self.tokenizer.decode(outputs[0], skip_special_tokens=False)
        response = self._parse_response(generated)
        return response

    def _parse_instruction(self, text):
        """Extract instruction from generated text."""
        try:
            # Find content after user header
            if "<|start_header_id|>user<|end_header_id|>" in text:
                parts = text.split("<|start_header_id|>user<|end_header_id|>")
                if len(parts) > 1:
                    instruction = parts[1]
                    # Remove end tokens
                    for end_token in ["<|eot_id|>", "<|start_header_id|>assistant"]:
                        if end_token in instruction:
                            instruction = instruction.split(end_token)[0]
                    return instruction.strip()
        except Exception as e:
            print(f"Parse error: {e}")
        return None

    def _parse_response(self, text):
        """Extract response from generated text."""
        try:
            # Find content after assistant header
            if "<|start_header_id|>assistant<|end_header_id|>" in text:
                parts = text.split("<|start_header_id|>assistant<|end_header_id|>")
                if len(parts) > 1:
                    response = parts[-1]  # Get the last assistant response
                    # Remove end tokens
                    for end_token in ["<|eot_id|>", "<|end_of_text|>"]:
                        if end_token in response:
                            response = response.split(end_token)[0]
                    return response.strip()
        except Exception as e:
            print(f"Parse error: {e}")
        return None

    def generate_pair(self):
        """Generate a single instruction-response pair."""
        instruction = self.generate_instruction()
        if instruction and len(instruction) > 10:
            response = self.generate_response(instruction)
            if response and len(response) > 10:
                return {
                    "instruction": instruction,
                    "response": response
                }
        return None

# Initialize generator
generator = MagpieGenerator(model, tokenizer, config)
print("Generator initialized!")

Generator initialized!


## 4. Test Generation

In [11]:
# Test single generation
print("Testing single generation...")
print("="*50)

test_pair = generator.generate_pair()
if test_pair:
    print(f"Instruction:\n{test_pair['instruction']}")
    print("\n" + "="*50 + "\n")
    print(f"Response:\n{test_pair['response']}")
else:
    print("Generation failed. Check model and templates.")

Testing single generation...
Instruction:
In this problem, you are the manager of a small bakery. You have two types of bakers, skilled and unskilled. Skilled bakers make loaves of bread faster than unskilled ones, but their labor costs are also higher. You want to determine the optimal number of skilled and unskilled bakers to hire to meet a demand of 120 loaves of bread per hour, given that a skilled baker produces 20 loaves per hour and an unskilled baker produces 15 loaves per hour.

Let the number of skilled bakers hired be denoted by $S$ and the number of unskilled bakers hired be denoted by $U$. The production rate per hour of skilled and unskilled bakers are $R_S$ and $R_U$ respectively, and the labor costs of skilled and unskilled bakers per hour are $C_S$ and $C_U$ respectively.

Production rate per hour of skilled and unskilled bakers can be represented by $R_S = 20$ loaves/hour and $R_U = 15$ loaves/hour.

Labor costs of skilled and unskilled bakers can be represented by $C

In [12]:
# Test multiple generations
print("Testing 5 generations...")
print("="*50)

for i in range(5):
    pair = generator.generate_pair()
    if pair:
        print(f"\n[{i+1}] Instruction: {pair['instruction'][:100]}...")
    else:
        print(f"[{i+1}] Failed")

Testing 5 generations...

[1] Instruction: ## Step 1: Understand the problem
The problem asks us to determine the value of the variable x from ...

[2] Instruction: Let f(x) = (6x + 1)/(3x - 5). Find the derivative of f(x) using the quotient rule and simplify your ...

[3] Instruction: Let $f(x) = x^4 + 4x + 2$ and $g(x) = 2x^4 + 6x^2 + x + 1$. Find $f(g(x))$....

[4] Instruction: ## Step 1: Understand the problem
To solve this problem, we need to find the sum of all the numbers ...

[5] Instruction: ## Step 1: Define the problem
We need to find the value of x such that when 1 is added to it and the...


## 5. Batch Generation with Checkpoints

In [13]:
# Checkpoint paths
DATA_DIR = f"{PROJECT_ROOT}/data/raw"
os.makedirs(DATA_DIR, exist_ok=True)

CHECKPOINT_PATH = f"{DATA_DIR}/instructions_checkpoint.json"
FINAL_PATH = f"{DATA_DIR}/instructions_raw.json"

# Load/Save functions
def save_checkpoint(data, path):
    """Save checkpoint to file."""
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"Saved {len(data)} samples to {path}")

def load_checkpoint(path):
    """Load existing checkpoint if available."""
    if os.path.exists(path):
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"Loaded {len(data)} samples from checkpoint")
        return data
    print("No checkpoint found. Starting fresh.")
    return []

print("Checkpoint functions defined!")

Checkpoint functions defined!


In [14]:
# Load existing checkpoint and calculate start index
generated_data = load_checkpoint(CHECKPOINT_PATH)
start_idx = len(generated_data)

print(f"\n{'='*50}")
print(f"📊 Generation Status")
print(f"{'='*50}")
print(f"✅ Completed: {start_idx} / {TARGET_SAMPLES}")
print(f"⏳ Remaining: {TARGET_SAMPLES - start_idx}")
print(f"💾 Checkpoint every: {CHECKPOINT_INTERVAL} samples")
if start_idx > 0:
    print(f"🚀 Resuming from index: {start_idx}")
else:
    print(f"🆕 Starting fresh")
print(f"{'='*50}\n")

Loaded 1500 samples from checkpoint

📊 Generation Status
✅ Completed: 1500 / 1500
⏳ Remaining: 0
💾 Checkpoint every: 20 samples
🚀 Resuming from index: 1500



In [15]:
# Generation settings
TARGET_SAMPLES = config['data_generation']['target_raw_samples']
CHECKPOINT_INTERVAL = config['data_generation']['checkpoint_interval']

print(f"Target samples: {TARGET_SAMPLES}")
print(f"Checkpoint interval: {CHECKPOINT_INTERVAL}")

Target samples: 1500
Checkpoint interval: 20


In [16]:
# Load existing checkpoint if available
generated_data = load_checkpoint(CHECKPOINT_PATH)
start_idx = len(generated_data)

print(f"Loaded {start_idx} existing samples")
print(f"Remaining: {TARGET_SAMPLES - start_idx} samples")

Loaded 1500 samples from checkpoint
Loaded 1500 existing samples
Remaining: 0 samples


In [ ]:
# Main generation loop
failed_count = 0
max_failures = 100  # Stop if too many consecutive failures

print(f"\nStarting generation from index {start_idx}...")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*50)

pbar = tqdm(total=TARGET_SAMPLES, initial=start_idx, desc="Generating")

while len(generated_data) < TARGET_SAMPLES:
    try:
        pair = generator.generate_pair()

        if pair:
            generated_data.append(pair)
            pbar.update(1)
            failed_count = 0

            # Save checkpoint with progress tracking
            if len(generated_data) % CHECKPOINT_INTERVAL == 0:
                save_checkpoint(generated_data, CHECKPOINT_PATH)
                progress_pct = (len(generated_data) / TARGET_SAMPLES) * 100
                print(f"✓ Checkpoint: {len(generated_data)}/{TARGET_SAMPLES} ({progress_pct:.1f}%)")

                # Note for A100 users about reduced checkpoint frequency
                if CHECKPOINT_INTERVAL >= 100:
                    print(f"   ℹ️  A100 optimization: Reduced checkpoint frequency minimizes disk I/O")

                # Clear GPU cache periodically
                gc.collect()
                torch.cuda.empty_cache()
        else:
            failed_count += 1
            if failed_count >= max_failures:
                print(f"\nToo many failures ({max_failures}). Stopping.")
                break

    except Exception as e:
        print(f"\nError: {e}")
        failed_count += 1
        if failed_count >= max_failures:
            break
        continue

pbar.close()
print(f"\nEnd time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total generated: {len(generated_data)} samples")

pbar.close()

# 1. Final Save (Securely saving to Google Drive)
final_path = CHECKPOINT_PATH.replace('checkpoint', 'final_full')
save_checkpoint(generated_data, final_path)

print("="*50)
print(f"✅ Final data saved to: {final_path}")
print(f"📅 End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📊 Total generated: {len(generated_data)} samples")

# ==================================================
# 2. Automatic Session Termination (Saves Compute Units)
# ==================================================
print("\n🚀 [Info] All tasks completed. Disconnecting the runtime immediately to save compute units.")

from google.colab import runtime
runtime.unassign() # This will disconnect the session and stop unit consumption


Starting generation from index 840...
Start time: 2025-12-24 11:42:39


Generating:  57%|█████▋    | 860/1500 [09:16<5:12:17, 29.28s/it]

Saved 860 samples to /content/drive/MyDrive/synthetic-instruction-tuner/data/raw/instructions_checkpoint.json
✓ Checkpoint: 860/1500 (57.3%)


Generating:  59%|█████▊    | 880/1500 [22:42<7:44:40, 44.97s/it]

Saved 880 samples to /content/drive/MyDrive/synthetic-instruction-tuner/data/raw/instructions_checkpoint.json
✓ Checkpoint: 880/1500 (58.7%)


...

Saved 1460 samples to /content/drive/MyDrive/synthetic-instruction-tuner/data/raw/instructions_checkpoint.json
✓ Checkpoint: 1460/1500 (97.3%)


Generating:  99%|█████████▊| 1480/1500 [6:22:34<12:11, 36.57s/it]

Saved 1480 samples to /content/drive/MyDrive/synthetic-instruction-tuner/data/raw/instructions_checkpoint.json
✓ Checkpoint: 1480/1500 (98.7%)


Generating: 100%|██████████| 1500/1500 [6:34:25<00:00, 35.86s/it]

Saved 1500 samples to /content/drive/MyDrive/synthetic-instruction-tuner/data/raw/instructions_checkpoint.json
✓ Checkpoint: 1500/1500 (100.0%)

End time: 2025-12-24 18:17:05
Total generated: 1500 samples
Saved 1500 samples to /content/drive/MyDrive/synthetic-instruction-tuner/data/raw/instructions_final_full.json
✅ Final data saved to: /content/drive/MyDrive/synthetic-instruction-tuner/data/raw/instructions_final_full.json
📅 End time: 2025-12-24 18:17:05
📊 Total generated: 1500 samples

🚀 [Info] All tasks completed. Disconnecting the runtime immediately to save compute units.


In [17]:
# Save final data
save_checkpoint(generated_data, FINAL_PATH)
print(f"\nFinal data saved to: {FINAL_PATH}")

Saved 1500 samples to /content/drive/MyDrive/synthetic-instruction-tuner/data/raw/instructions_raw.json

Final data saved to: /content/drive/MyDrive/synthetic-instruction-tuner/data/raw/instructions_raw.json


## 6. Data Statistics

In [18]:
# Calculate statistics
import numpy as np

instruction_lengths = [len(d['instruction'].split()) for d in generated_data]
response_lengths = [len(d['response'].split()) for d in generated_data]

print("=" * 50)
print("DATA STATISTICS")
print("=" * 50)
print(f"Total samples: {len(generated_data)}")
print(f"\nInstruction length (words):")
print(f"  Mean: {np.mean(instruction_lengths):.1f}")
print(f"  Min: {np.min(instruction_lengths)}")
print(f"  Max: {np.max(instruction_lengths)}")
print(f"  Median: {np.median(instruction_lengths):.1f}")
print(f"\nResponse length (words):")
print(f"  Mean: {np.mean(response_lengths):.1f}")
print(f"  Min: {np.min(response_lengths)}")
print(f"  Max: {np.max(response_lengths)}")
print(f"  Median: {np.median(response_lengths):.1f}")

DATA STATISTICS
Total samples: 1500

Instruction length (words):
  Mean: 155.6
  Min: 9
  Max: 229
  Median: 169.0

Response length (words):
  Mean: 169.7
  Min: 4
  Max: 434
  Median: 157.0


In [19]:
# Show sample data
print("\n" + "=" * 50)
print("SAMPLE DATA")
print("=" * 50)

for i, sample in enumerate(random.sample(generated_data, min(3, len(generated_data)))):
    print(f"\n--- Sample {i+1} ---")
    print(f"Instruction: {sample['instruction'][:200]}...")
    print(f"Response: {sample['response'][:200]}...")


SAMPLE DATA

--- Sample 1 ---
Instruction: ## Step 1: Understand the given equation
We are given the equation $2 \sin^2 x + 4 \cos^2 x + 3 \sin x = 4 \cos^2 x + 3$.

## Step 2: Apply the Pythagorean identity
We know that $\sin^2 x + \cos^2 x =...
Response: ## Step 5: Simplify the equation further
Expanding the equation further, we get $2 \sin^2 x + 4 - 4 \sin^2 x + 3 \sin x = 4 - 4 \sin^2 x + 3$.

## Step 6: Combine like terms
Combining like terms, we g...

--- Sample 2 ---
Instruction: ## Step 1: Understand the problem and the given information.
The problem is asking us to find the value of x in the equation 5x + 3y = 11, where we are given that the value of x is negative and that t...
Response: ## Step 1: Understand the problem and the given information.
We are given a linear equation 5x + 3y = 11 and a slope of 2. We need to find the value of x given that it is negative.

## Step 2: Express...

--- Sample 3 ---
Instruction: Let $\{a_n\}$ and $\{b_n\}$ be two sequences defined by th

## 7. Cleanup

In [20]:
# Free GPU memory
del model
del tokenizer
del generator
gc.collect()
torch.cuda.empty_cache()

print("Memory cleared!")

Memory cleared!


## ✅ Generation Complete!

### Summary:
- Generated instruction-response pairs saved to `data/raw/instructions_raw.json`
- Checkpoint available at `data/raw/instructions_checkpoint.json`

### Next Steps:
1. Proceed to `03_quality_filtering.ipynb` for data filtering
2. Target: Filter 1,500 → ~1,000 high-quality samples
3. Optimized size allows more focus on evaluation and comparative analysis (realistic for Colab free tier)